In [46]:
import geopandas as gp
import pandas as pd
import os
import numpy as np
import json

In [47]:
# Functions

def format_df_from_csv(df): # Takes df read from raw USDA CSV and kills off whitespace and other bs 
    df.columns = [t.strip() for t in list(df)]
    df.columns = df.columns.str.replace('\s+','')  # remove whitespace from headers
    df.columns = df.columns.str.lower()
    df['countycode'] = df['countycode'].apply(lambda x: str(x).zfill(3)) # Zpad the county codes to match the shps
    df.county = df.county.str.strip()
    if "commodity" in df.columns:
        df.rename(columns={'commodity': 'commoditycode'}, inplace = True)
    return df

def filter_by_county(df,county):
    if county is type(int):
        g = df[df.countycode == county]
    else:
        county = county.lower()
        df.county = df.county.str.replace('\s+','')
        df.county = df.county.str.lower()
        fin_df = df[df.county == county]
    return(fin_df)

def drop_crops(df):
    allcats = df.cropname
    badfields = ["HORSE","WOOL", "MILK", "LIVESTOCK", "CATTLE", "POULTRY", "PIG","MILK", "APIARY", "SERVICE", "BEES", "PASTURE", "SILAGE","SHEEP", "FEED","FLOWERS","NURSERY", "OSTRICH", "TURKEYS", "CHICKENS", "MANURE", "GOATS", "LAMBS", "BIRDS"]
    badfields = ['BEES', 'OSTRICH' ,'SERVICE' ,'HORSE' ,'PASTURE' ,'POULTRY' ,'MILK' ,'TURKEY', 'WOOL' ,'APIARY' ,'RABBITS', 'SHEEP', 'BIOMASS' ,'CATTLE' ,'CHICKEN', 'TREE', 'EGGS' ,'FISH' ,'FLOWER' ,'FOREST' ,'BIRD','GOAT' ,'GAME', 'HOGS' ,'PIG' ,'LAMB' ,'LIVESTOCK' ,'MANURE','NURSERY']
    keep = [x for x in allcats if x.split() not in  badfields]
    result = [r for r in allcats if not any(z in r for z in badfields)]
    df = df.loc[df['cropname'].isin(result)]
    return df

def replace_nans_with_prod(df):
    crops = df.cropname.unique()
    cdfs = []
    for c in crops:
        f = df.loc[df.cropname== c]
        
        f.production = pd.to_numeric(f.production, errors='coerce')
        f.value = pd.to_numeric(f.value, errors='coerce')
        f.countycode = pd.to_numeric(f.countycode, errors = "coerce")
        f.loc[f.production.isnull(), 'production'] = f.loc[f.production.isnull()].value.astype(np.float)*f.loc[f.production.isnull()].value.astype(np.float) / f[f.countycode == 999].value.values.astype(np.float)

        cdfs.append(f)
    
    fin_df = pd.concat(cdfs)
    return(fin_df)

def calc_prod(df):
    totals = df[df.countycode == 999]
    crops = df.cropname.unique()
    cdfs = []
    for c in crops:
        f = df.loc[df.cropname== c]
        
        f.production = pd.to_numeric(f.production, errors='coerce')
        f.value = pd.to_numeric(f.value, errors='coerce')
        f.countycode = pd.to_numeric(f.countycode, errors = "coerce")
        f.loc[f.production.isnull(), 'production'] = f.loc[f.production.isnull()].value.astype(np.float)*f.loc[f.production.isnull()].value.astype(np.float) / totals.loc[totals.cropname==c].value.astype(np.float)

        cdfs.append(f)
    
    fin_df = pd.concat(cdfs)
    return(fin_df)

def sum_production(df):
    result = {}
    year = df.year
    df = df.production.dropna(axis=0, how='all')
    crop_sum = []
    for i in df:
        try:
            crop_sum.append(float(i))
        except:
            continue    
            
    return sum(crop_sum)

def write_csv(df):
    outfn = os.path.join(os.getcwd(),str(df['year']) + "_y")
    df.to_csv(out_fn, sep='\t')
    return(outfn)

def process_county(county):
    
    # Read the USDA csvs as dfs and format col names
    fdir = os.path.join(os.getcwd(), "csvs")
    files = [os.path.join(fdir,x) for x in os.listdir(fdir)]
    tables = [pd.read_csv(x) for x in files]
    dfs = []
    
    for t in tables:
        dfs.append(format_df_from_csv(t))
    
    # drop bad crops from shapefiles
    cleaned = []
    for i in dfs:
        cleaned.append(drop_crops(i))
    
    # Check for funky commoditites and remove them (some commodities are blank but still have production values)
    for i in cleaned:
        i = i[i.commoditycode > 100000] 

    # Calculate production for crops missing that field
    filled = []
    for i in cleaned:
        filled.append(replace_nans_with_prod(i))
    
    # just do tulare
    counties = []
    for i in filled:
        counties.append(filter_by_county(i,county))
    
    # build the output dict, y parameter vector 
    y = {}
    for i in counties:
        year = int(filled[0].year.mode())
        year = str(year)
        prod = sum_production(i)
        y[year] = prod
    
    return (y)


In [13]:
fdir = os.path.join(os.getcwd(), "csvs")
files = [os.path.join(fdir,x) for x in os.listdir(fdir)]
tables = [pd.read_csv(x) for x in files]
dfs = []
    
for t in tables:
    dfs.append(format_df_from_csv(t))

In [67]:
s = "el   dorado".strip()
s = s.replace(' ','')
s

'eldorado'

In [15]:
county = "tulare"
by_county = []
for i in dfs:
    by_county.append(filter_by_county(i,county))

In [16]:
cleaned = []
for i in by_county:
    cleaned.append(drop_crops(i))

In [17]:
# Check for funky commoditites and remove them (some commodities are blank but still have production values)
for i in cleaned:
    i = i[i.commoditycode > 100000] 

In [19]:
filled = []
for i in cleaned:
    filled.append(calc_prod(i))

In [22]:
p = sum_production(filled[0])

In [42]:
year = int(filled[0].year.mode())
year = str(year)
year

'2009'

In [48]:
def process_county(county):
    fdir = os.path.join(os.getcwd(), "csvs")
    files = [os.path.join(fdir,x) for x in os.listdir(fdir)]
    tables = [pd.read_csv(x) for x in files]
    dfs = []

    for t in tables:
        dfs.append(format_df_from_csv(t))

    by_county = []
    for i in dfs:
        by_county.append(filter_by_county(i,county))

    cleaned = []
    for i in by_county:
        cleaned.append(drop_crops(i))

    # Check for funky commoditites and remove them (some commodities are blank but still have production values)
    for i in cleaned:
        i = i[i.commoditycode > 100000] 

    filled = []
    for i in cleaned:
        filled.append(calc_prod(i))

    y = {}
    for i in filled:
        year = int(i.year.mode())
        year = str(year)
        y[year] = sum_production(i)

    return y

    '''
    with open(county+'.txt', 'w') as file:
        file.write(json.dumps(y))
    '''

In [49]:
county = "tulare"
process_county(county)

In [ ]:
lists = sorted(tulare.items()) # sorted by key, return a list of tuples

x, y = zip(*lists)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(x[9:],y[9:])
plt.show()